In [1]:
import tensorflow as tf
from datetime import datetime
# linear regression
import numpy as np
from sklearn.datasets import fetch_california_housing

In [2]:
# get the housing data
housing=fetch_california_housing()
m,n=housing.data.shape
housing_data_plus_bias=np.c_[np.ones((m,1)),housing.data]

In [7]:
m

20640

In [3]:
# define a function to fetch a batch of data from the original set
def fetch_batch(X_train,y_train,batch_index,batch_size):
    start=batch_index*batch_size
    end=start+batch_size
    X_batch=X_train[start:end]
    y_batch=y_train[start:end]
    return X_batch,y_batch

In [21]:
# with a saver node
from sklearn.utils import shuffle
from tqdm import tqdm
import os
from sklearn.preprocessing import scale

#hyper parameters
n_epochs=1000
learning_rate=0.01
batch_size=m

# tf model construction
tf.reset_default_graph()
X_train = scale( housing_data_plus_bias, axis=0, with_mean=True, with_std=True, copy=True )
y_train = housing.target.reshape(-1,1)
X=tf.placeholder(tf.float32,shape=(None,n+1),name="X")
y=tf.placeholder(tf.float32,shape=(None,1),name="y")
theta=tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name="theta")
y_pred=tf.matmul(X,theta,name="predictions")
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name="mse")
mse_summary=tf.summary.scalar('MSE',mse)
optimizer=tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)
training_op=optimizer.minimize(mse)
init=tf.global_variables_initializer()

# log writer
now=datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir="tf_logs"
logdir="{}/run-{}/".format(root_logdir,now)
file_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())

# call tf session

with tf.Session() as sess:
    sess.run(init)
    n_batches=int(np.ceil(m/batch_size))
    for epoch in tqdm(range(n_epochs)):
        X_train,y_train=shuffle(X_train,y_train)
        for batch_index in range(n_batches):
            X_batch,y_batch=fetch_batch(X_train,y_train,batch_index,batch_size)
            if batch_index % 1 ==0:
                summary_str=mse_summary.eval(feed_dict={X:X_batch,y:y_batch})
                step=epoch*n_batches+batch_index
                file_writer.add_summary(summary_str,step)
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
    best_theta=theta.eval()
print(best_theta)

100%|██████████| 1000/1000 [00:03<00:00, 292.72it/s]


[[ 0.5041466 ]
 [ 0.8296212 ]
 [ 0.11875194]
 [-0.26553035]
 [ 0.3056991 ]
 [-0.00450293]
 [-0.03932634]
 [-0.89988196]
 [-0.8705375 ]]


In [14]:
step

999

In [ ]:
file_writer.close()